In [1]:
import datetime
print "Created on: {} ".format(datetime.datetime.now().strftime('%Y-%m-%d %H:%M'))

Created on: 2017-08-01 01:53 


# Nine: Review, Revamp, Overhaul
All the way up to [Entry 8](http://cliffchew84.github.io/eight_new_data_source.html), I have been doing various components of the data science workflow. However, after gaining some experience, I have decided to overhaul my analysis. In particular, when trying to move forward, I find myself struggling to fully understand what I have done previously. This could unfortunately, be due to my inability to put consistent efforts in my side project. Hopefully, by spending an extra few hours going through my different tables, and including more comments, I can give myself more ability to move faster. 

1. **Player statistics:** I always had the plan of including player statistics to improve my analysis. 
1. **Changing data source:** However, because my current data source had many issues, I am using a different data source which has player_ids for me to match.
1. **Layout:** This is more symbolic than significant, but I decide to change my notebook layout to signify the change of my data set. 

In [2]:
import os
import glob
import dropbox
import warnings
import datetime
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
base_folder = os.path.abspath(".")

In [3]:
os.chdir(base_folder)
os.chdir(os.getcwd() + "/new_data")

### Processes and Workflow
**From my raw data...** I have a ton of restructuring and processing to perform on my new data set. To allow easier iterations of the code through the different seasons, I have made many of my processing steps into functions, with each function focusing on 1 or 2 processes.

### Breaking processing bottlenecks
**I learned that this is the key bottleneck of my current workflow**, and is slowly down my iterations on creating newer variables to test my models. Hence, one other focus of this part of my workflow is to try to properly identify each function / process, and make them as modular and flexible as possible, so that I can easily work new feature engineerings into my workflow to create new variables. 

### Loading all files
I have 5 files that are loaded.
1. **home_csv -->** Allows me to indicate home and away!
2. **team_csv -->** Team statistics (Team level)
3. **more_stats_csv -->** Advanced Team Statistics (from 1996, currently not in use, Team level) 
4. **game_date_csv -->** Game dates and potential attendance (Team level)
5. **win_loss_csv**

## Main Script

In [4]:
def load_all_files(year):
    home_csv = pd.read_csv("more_home_away_{}.csv".format(year))
    teams_csv = pd.read_csv("team_{}.csv".format(year), usecols = [
            'GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 
            'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS'])

    # This has issues... Need to think if dropping these numbers are better
    more_stats_csv = pd.read_csv("more_team_stats_{}.csv".format(year))
    game_date_csv = pd.read_csv("game_date_{}.csv".format(year))

    # This has issues too... Need to re-calculate them myself...
    win_loss_csv = pd.read_csv("more_wins_losses_{}.csv".format(year)) 
    return home_csv, teams_csv, more_stats_csv, game_date_csv, win_loss_csv

#### **Create Variable:** HOME/AWAY variable for each team

In [5]:
def create_home_away_var(table):
    """Create home or away variable. Provides """
    home = table[["GAME_ID", "HOME_TEAM_ID", "SEASON", "LIVE_PERIOD"]].rename(columns={"HOME_TEAM_ID":"TEAM_ID"})
    home["Home"] = "Home"

    away = table[["GAME_ID", "VISITOR_TEAM_ID", "SEASON", "LIVE_PERIOD"]].rename(columns={"VISITOR_TEAM_ID":"TEAM_ID"})
    away["Home"] = "Away"
    return home.append(away)

#### **Create Variable:** Convert GAME_DATE to datetime

In [6]:
def create_date_variable(main_table):
    game_date = main_table.copy() 
    game_date["GAME_DATE"] = [i.split(", ", 1)[1] for i in game_date.GAME_DATE]
    game_date = game_date[["GAME_DATE", "GAME_ID"]]
    game_date["GAME_DATE"] = [datetime.datetime.strptime(i, "%B %d, %Y").date() for i in game_date["GAME_DATE"]] 
    return game_date

#### Add game statistics 

In [7]:
def add_game_stats(table, table1):
    """ Creates Game Count variable! Needs to create home-away variable first! """
    return pd.merge(table, table1, on=["TEAM_ID", "GAME_ID"])

#### Creates Game counter variable.

In [8]:
def create_game_count_var(input_table):
    """ Creates Game Count variable! Needs to create home-away variable first! """
    input_table["G"] = 1
    input_table["G"] = input_table.sort_values(["TEAM_NAME", "GAME_DATE"]).groupby("TEAM_ID")["G"].transform(pd.Series.cumsum)
    return input_table.sort_values(["TEAM_ID", "G"])

#### Creates a variable for number of days from previous game 

In [9]:
def create_days_from_previous_games_var(main_table):
    main_table['p_games'] = main_table["GAME_DATE"] - main_table["GAME_DATE"].shift(1)
    main_table["p_games"] = [str(i).split(" ")[0] for i in main_table["p_games"]]
    main_table["p_games"] = pd.to_numeric(main_table["p_games"], errors=coerce)
    main_table.loc[main_table["p_games"] < 0, "p_games"] = np.nan
    return main_table

#### Function to stack competing team stats side by side!
- Currently, the stats are stored by per team in a single table. However, I each row in the table to reflect each game, with home and away game stats shown in 1 row to faciliate my analysis. 

In [10]:
def create_opp_stats(table):
    """ Function duplicates all games, and merges them to form home and away records in each row. 
        This directly doubles the number of games stats that I have in the table, as now I have 
        both team_a and team_b from each game merged side by side. s"""
    opp_merge = table.copy()
    opp_merge.Home = table.Home.replace("Home", "Away_2").replace("Away", "Home").replace("Away_2", "Away")
    
    return pd.merge(table.drop(["PLUS_MINUS", "SEASON", "LIVE_PERIOD"], axis=1), 
                    opp_merge.drop(["PLUS_MINUS"], axis=1), 
                    on=["Home", "GAME_ID", "GAME_DATE"], how="left")

#### Create Win-Loss variables!
- Currently, I am just focusing on predicting win-loss of each game, base on home and away team statistics. This is the function that creates my win-loss variable. **Future iterations of my prediction models can consider for estimating expected points scored by the home and away team**, and using those estimates to calculate the points spread between the 2 teams. Would definitely be very intereting to see which model gives better predictions!

In [11]:
def create_win_loss_vars(main_table):
    main_table["WL_x"] = 0
    main_table["W_x"] = 0
    main_table["L_x"] = 0
    main_table["W_y"] = 0
    main_table["L_y"] = 0
    main_table.loc[main_table["PTS_x"] > main_table["PTS_y"], "WL_x"] = 1
    main_table.loc[main_table["PTS_x"] > main_table["PTS_y"], "W_x"] = 1
    main_table.loc[main_table["PTS_x"] < main_table["PTS_y"], "L_x"] = 1
    main_table.loc[main_table["PTS_x"] < main_table["PTS_y"], "W_y"] = 1
    main_table.loc[main_table["PTS_x"] > main_table["PTS_y"], "L_y"] = 1
    return main_table

#### Creation of average and shooting percentage statistics
- **Per game statistics:** Divide cumulative game stats by total games thus far (Requires n-th games played in season variable).
- **Shooting percentage statistics:** Cumulatively divide total shot made by total shots attempted.

#### Averages!
**Important note:** The initial table are built on the stats from team_x and team_y (aka team_x_opp). Hence, calculation of averages have to be done on the cumulated game_dates from **G_x** only! *__Counter example is when I previously used G_y to calculate the averages for team_y, and where team_y was on G_y == 2, but team_x is on G_x == 1. Hence, the numbers to represent how strong team_x has been against their opponents only had 1 game, but I actually used 2 games.__*

In [12]:
def create_averages(table_return, variables):
    """ Averages are done by cumulatively summing the stats by games, and then dividing by no of games!
        I will directly over-write the columns to make the table manageable."""
    table = table_return.copy()
    table[[i + "_x" for i in variables]] = table[
        [i + "_x" for i in variables]].astype('float').div(table['G_x'].astype('float'),axis='index')
    
    table[[i + "_y" for i in variables]] = table[
        [i + "_y" for i in variables]].astype('float').div(table['G_x'].astype('float'),axis='index')
    
    return table

#### Creating Shooting Percentages!

In [13]:
def create_shooting_percentage_vars(main_table):
    for i in ["y", "x"]:
        main_table["FGP_" + i] = main_table["FGM_" + i] / main_table["FGA_" + i]
        main_table["FG3P_" + i] = main_table["FG3M_" + i] / main_table["FG3A_" + i]
        main_table["FTP_" + i] = main_table["FTM_" + i] / main_table["FTA_" + i]
    return main_table

#### NBA domain-knowledge stats
- I am also adding NBA domain-knowledge stats (for a lack of a better name) to see how much they can help with my prediction scores. 

1. **Efficiency ratings: EFG% = (FGM + (0.5 * 3PM)) / FGA **

In [14]:
def create_efg_var(main_table):
    main_table["EFG_x"] = (main_table["FGM_x"] + (.5 * main_table["FG3M_x"])) / main_table["FGA_x"]
    main_table["FGP_x"] = main_table["FGM_x"] / main_table["FGA_x"]
    
    main_table["EFG_y"] = (main_table["FGM_y"] + (.5 * main_table["FG3M_y"])) / main_table["FGA_y"]
    main_table["FGP_y"] = main_table["FGM_y"] / main_table["FGA_y"]
    
    return main_table

1. **Free-throw attempts to Field-goal attempts ratio**

In [15]:
def create_fta_to_fga_ratio(main_table):
    main_table["FTA_FGA_x"] = main_table["FTA_x"] / main_table["FGA_x"]
    main_table["FTA_FGA_y"] = main_table["FTA_y"] / main_table["FGA_y"]
    return main_table

2. **Offensive rebounding percentage (OREB%) = Offensive rebounds / (Offensive rebounds + Opponent defensive rebounds)**
3. **Defensive rebounding percentage (DREB%) = Defensive rebounds / (Defensive rebounds + Opponent offensive rebounds)**

In [16]:
def create_rebs_efficiency_vars(main_table):
    main_table["oreb_p_x"] = main_table["OREB_x"] / (main_table["OREB_x"] + main_table["DREB_y"])
    main_table["dreb_p_x"] = main_table["DREB_x"] / (main_table["DREB_x"] + main_table["OREB_y"])

    main_table["oreb_p_y"] = main_table["OREB_y"] / (main_table["OREB_y"] + main_table["DREB_x"])
    main_table["dreb_p_y"] = main_table["DREB_y"] / (main_table["DREB_y"] + main_table["OREB_x"])
    return main_table

**Filter for variables that I want:** Shifting the stats so predictions can be based on past cumulated data

In [17]:
def shift_game_stats_down_by_one(main_table):
    """ Shifting the games stats down by one (for each team each season) 
        so that predictions can be based on past cumulated data."""
    base_shift = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 
                  'TO', 'PF', 'PTS', 'W', 'L', "EFG", "FGP", "oreb_p", "dreb_p", "FG3P", "FTP"]

    vars_to_shift = [i + "_x" for i in base_shift] + [i + "_y" for i in base_shift] + ["FTA_FGA_x", "FTA_FGA_y"]
    final_shift = main_table.copy()
    final_shift[vars_to_shift] = final_shift[vars_to_shift].shift(1)
    return final_shift[(final_shift["G_x"] > 1)] 

#### **Base table is on TEAM_A:** 
- Base table has team_a (x) and team_a_opp (y) stats. Duplicating from the same table, the final step is adding **team_b** and **team_b opponents** to the base table!
- **(1) final_shift is the base (2) final_shift_opp is the appending table.**
- From final_shift, remove ta_opp stuff (because those will come from final_shift_opp)
- From final_shift_opp, remove **"WL_tb"** and **"Home"**, because base is on final_shift!

In [18]:
def create_team_ab_and_opp_table(final_shift):
    # Create dup table
    final_shift_opp = final_shift.copy()
    
    # Rename team_x and team_y to team_a and team_a_opponents
    final_shift.columns = [i.replace("_x", "_ta").replace("_y", "_ta_opp") for i in final_shift.columns]
    
    # Rename team_x and team_y to team_b and team_b_opponents
    final_shift_opp.columns = [i.replace("_x", "_tb").replace("_y", "_tb_opp") for i in final_shift_opp.columns]
    
    # Renaming final_shift_opp so that I can merge the 2 tables the same IDs... 
    final_shift_opp.rename(columns={"TEAM_ID_tb_opp":"TEAM_ID_ta", 
                                    "TEAM_NAME_tb_opp":"TEAM_NAME_ta"}, 
                           inplace=True)
    
    ffinals = pd.merge(final_shift.drop(["TEAM_ID_ta_opp", "TEAM_NAME_ta_opp"], axis=1), 
                       final_shift_opp.drop(["WL_tb", "Home", "SEASON", "LIVE_PERIOD"], axis=1),           
                       on=["GAME_ID", "TEAM_ID_ta", "TEAM_NAME_ta", "GAME_DATE"])
    return ffinals

#### Keep only Home 

In [19]:
def filter_home_teams(main_table):
    finals_home = main_table[main_table["Home"] == "Home"]
    finals_home.columns = [i.lower() for i in finals_home.columns]
    finals_home.rename(columns={"w_ta":"w_rate_ta", "w_tb":"w_rate_tb"}, inplace=True)
    return finals_home

**Finally...** all the functions that I currently want to perform are settled! Now it's just executing them on the actual the full dataset!

In [20]:
final_table = pd.DataFrame()
for i in range(1986, 2016):
    home_csv, teams_csv, more_stats_csv, game_date_csv, win_loss_csv = load_all_files(i)
    
    # The base is created from HOME_AWAY variable...
    merge_home_away = create_home_away_var(home_csv).drop_duplicates()
    
    # And then adding GAME_DATE to the base 
    merge_date = pd.merge(merge_home_away, create_date_variable(game_date_csv))

    # Then adding GAME_STATS to the base
    merge_team_stats = add_game_stats(teams_csv.drop_duplicates(), merge_date)

    # Then cumulating GAME_COUNTER to the base
    merge_game_count = create_game_count_var(merge_team_stats)

    # Then adding DAYS_FROM_PREVIOUS_GAME to the base
    merge_previous_date = create_days_from_previous_games_var(merge_game_count)

    # As the tables are in one continous table, I have to MERGE their STATS SIDE_BY_SIDE to form team_a vs team_b
    # This creates the first explosion of variables in the base table
    merge_opp = create_opp_stats(merge_previous_date)

    # Create WIN_LOSS variable by comparing team_a vs team_b points
    # This can only be done when I have both teams data aligned side by side, aka the execution of create_opp_stats
    merge_opp = create_win_loss_vars(merge_opp)
    
    # Variables to accumulate 
    """ I need to cumulate all numeric stats """
    var_to_accum = merge_opp.drop(["GAME_ID", "TEAM_ID_x", "TEAM_ID_y", "TEAM_NAME_x", "TEAM_NAME_y", 
                                   "G_x", "G_y", "Home", "GAME_DATE", "WL_x", 'p_games_x', 'p_games_y', 
                                   "SEASON", "LIVE_PERIOD"], axis=1).columns
    
    """ Potential for change: Right now, I am using the entire season for the stats calculations. 
        However, it might be more accurate to use rolling window calculations instead."""
    # Create accumulative variables
    merge_total = merge_opp.sort_values(["TEAM_NAME_x", "GAME_DATE"])
    merge_total[var_to_accum] = merge_total.groupby('TEAM_ID_x')[var_to_accum].transform(pd.Series.cumsum)

    # Create NBA domain-knowledge variables
    merge_total = create_rebs_efficiency_vars(
        create_fta_to_fga_ratio(
            create_efg_var(merge_total)))

    # Create percentage variables 
    """Dividing the cumulative stats of these variables by the games_played thus far to get their averages."""
    vars_to_average = ["OREB", "DREB", "REB", "AST", "STL", "BLK", "TO", "PF", "PTS", "W", "L"]
    merge_total_ave = create_shooting_percentage_vars(
        create_averages(merge_total, vars_to_average))

    merge_games_shifted = shift_game_stats_down_by_one(merge_total_ave)
    final_t_ab_opp = create_team_ab_and_opp_table(merge_games_shifted)
    finals_home = filter_home_teams(final_t_ab_opp)
    print("Season {} Done".format(i))
    
    final_table = final_table.append(finals_home)

Season 1986 Done
Season 1987 Done
Season 1988 Done
Season 1989 Done
Season 1990 Done
Season 1991 Done
Season 1992 Done
Season 1993 Done
Season 1994 Done
Season 1995 Done
Season 1996 Done
Season 1997 Done
Season 1998 Done
Season 1999 Done
Season 2000 Done
Season 2001 Done
Season 2002 Done
Season 2003 Done
Season 2004 Done
Season 2005 Done
Season 2006 Done
Season 2007 Done
Season 2008 Done
Season 2009 Done
Season 2010 Done
Season 2011 Done
Season 2012 Done
Season 2013 Done
Season 2014 Done
Season 2015 Done


In [21]:
final_table.shape

(33801, 114)

In [22]:
final_table[final_table.game_id.duplicated()].season.value_counts()

Series([], Name: season, dtype: int64)

In [24]:
final_table.head()

,game_id,team_id_ta,team_name_ta,fgm_ta,fga_ta,fg3m_ta,fg3a_ta,ftm_ta,fta_ta,oreb_ta,dreb_ta,reb_ta,ast_ta,stl_ta,blk_ta,to_ta,pf_ta,pts_ta,home,game_date,g_ta,p_games_ta,fgm_ta_opp,fga_ta_opp,fg3m_ta_opp,fg3a_ta_opp,ftm_ta_opp,fta_ta_opp,oreb_ta_opp,dreb_ta_opp,reb_ta_opp,ast_ta_opp,stl_ta_opp,blk_ta_opp,to_ta_opp,pf_ta_opp,pts_ta_opp,season,live_period,g_ta_opp,p_games_ta_opp,wl_ta,w_rate_ta,l_ta,w_ta_opp,l_ta_opp,efg_ta,fgp_ta,efg_ta_opp,fgp_ta_opp,fta_fga_ta,fta_fga_ta_opp,oreb_p_ta,dreb_p_ta,oreb_p_ta_opp,dreb_p_ta_opp,fg3p_ta_opp,ftp_ta_opp,fg3p_ta,ftp_ta,team_id_tb,team_name_tb,fgm_tb,fga_tb,fg3m_tb,fg3a_tb,ftm_tb,fta_tb,oreb_tb,dreb_tb,reb_tb,ast_tb,stl_tb,blk_tb,to_tb,pf_tb,pts_tb,g_tb,p_games_tb,fgm_tb_opp,fga_tb_opp,fg3m_tb_opp,fg3a_tb_opp,ftm_tb_opp,fta_tb_opp,oreb_tb_opp,dreb_tb_opp,reb_tb_opp,ast_tb_opp,stl_tb_opp,blk_tb_opp,to_tb_opp,pf_tb_opp,pts_tb_opp,g_tb_opp,p_games_tb_opp,w_rate_tb,l_tb,w_tb_opp,l_tb_opp,efg_tb,fgp_tb,efg_tb_opp,fgp_tb_opp,fta_fga_tb,fta_fga_tb_opp,oreb_p_tb,dreb_p_tb,oreb_p_tb_opp,dreb_p_tb_opp,fg3p_tb_opp,ftp_tb_opp,fg3p_tb,ftp_tb
0,28600021,1610612755,76ers,43.0,84.0,2.0,3.0,20.0,30.0,18.000000,30.000000,48.000000,26.000000,7.000000,5.00,25.000000,27.000000,108.000000,Home,1986-11-02,2,2.0,39.0,83.0,0.0,3.0,26.0,35.0,14.000000,21.0,35.000000,26.000000,14.000000,4.000000,14.000000,26.000000,104.000000,1986,4,2,1.0,0,1.000000,0.000000,0.000000,1.000000,0.523810,0.511905,0.469880,0.469880,0.357143,0.421687,0.461538,0.681818,0.318182,0.538462,0.000000,0.742857,0.666667,0.666667,1610612737,Hawks,50.0,92.0,2.0,3.0,29.0,39.0,16.000000,27.000000,43.000000,27.000000,10.000000,2.000000,22.000000,35.000000,131.000000,2,1.0,38.0,77.0,0.0,3.0,35.0,42.0,9.000000,21.000000,30.000000,17.000000,5.000000,5.000000,22.000000,31.000000,111.000000,2,2.0,1.000000,0.000000,0.000000,1.000000,0.554348,0.543478,0.493506,0.493506,0.423913,0.545455,0.432432,0.750000,0.250000,0.567568,0.000000,0.833333,0.666667,0.743590
2,28600037,1610612755,76ers,134.0,253.0,4.0,11.0,70.0,92.0,14.333333,31.000000,45.333333,30.000000,8.000000,9.00,23.000000,26.666667,114.000000,Home,1986-11-05,4,1.0,136.0,290.0,3.0,13.0,76.0,99.0,18.666667,25.0,43.666667,27.333333,13.000000,2.333333,14.000000,26.333333,117.000000,1986,4,4,1.0,1,0.333333,0.666667,0.666667,0.333333,0.537549,0.529644,0.474138,0.468966,0.363636,0.341379,0.364407,0.624161,0.375839,0.635593,0.230769,0.767677,0.363636,0.760870,1610612749,Bucks,119.0,258.0,12.0,26.0,79.0,95.0,11.333333,29.333333,40.666667,21.666667,9.333333,5.333333,12.333333,27.000000,109.666667,4,1.0,116.0,262.0,0.0,11.0,84.0,102.0,17.333333,32.333333,49.666667,21.000000,6.333333,5.333333,17.333333,25.666667,105.333333,4,1.0,0.666667,0.333333,0.333333,0.666667,0.484496,0.461240,0.442748,0.442748,0.368217,0.389313,0.259542,0.628571,0.371429,0.740458,0.000000,0.823529,0.461538,0.831579
3,28600048,1610612755,76ers,170.0,329.0,4.0,12.0,105.0,135.0,13.000000,31.750000,44.750000,29.250000,9.250000,10.25,20.250000,25.250000,112.250000,Home,1986-11-07,5,2.0,177.0,382.0,5.0,20.0,93.0,124.0,19.250000,27.0,46.250000,27.250000,11.750000,3.000000,15.000000,26.750000,113.000000,1986,4,5,2.0,1,0.500000,0.500000,0.500000,0.500000,0.522796,0.516717,0.469895,0.463351,0.410334,0.324607,0.325000,0.622549,0.377451,0.675000,0.250000,0.750000,0.333333,0.777778,1610612759,Spurs,170.0,361.0,4.0,17.0,84.0,115.0,12.750000,25.000000,37.750000,27.750000,9.250000,2.250000,13.250000,23.500000,107.000000,5,2.0,183.0,347.0,2.0,11.0,85.0,114.0,13.250000,32.000000,45.250000,29.000000,6.500000,5.250000,17.750000,23.500000,113.250000,5,2.0,0.500000,0.500000,0.500000,0.500000,0.476454,0.470914,0.530259,0.527378,0.318560,0.328530,0.284916,0.653595,0.346405,0.715084,0.181818,0.745614,0.235294,0.730435
9,28600108,1610612755,76ers,423.0,867.0,10.0,32.0,257.0,334.0,13.200000,30.600000,43.800000,25.200000,10.100000,7.00,17.700000,25.400000,111.300000,Home,1986-11-19,11,3.0,429.0,888.0,10.0,45.0,243.0,312.0,14.500000,29.8,44.300000,27.900000,10.2000

### Some summary notes:
- I did my processing at a per season level, because there were some differences in the available data across the different seasons. Hence, although this requires me to slowly iterate the processing through each season, this approach gave me greater flexibility to operate with.
- For the sake of analysis, I felt that I should now separate regular season and playoff games in my analysis for now, as they are of very different nature.
- I found some missing games in my data (about 1 in every few seasons), and I think it has to do with how I did my data collection, and how the data is stored on the site. I will have consider on how to improve my data collection method in the near future as well. 
- **A key reason for refactoring my code is have a flexible workflow to inject player and team specific data**.

### More on adding more variables...
- Currently, my idea of adding more features is to merge by the respective game and player IDs that already exist in my main table. A different approach would be to weave my new features into the existing workflow, but I will have to constantly tear apart my existing workflow, and it will also make it very difficult for me to check my data integrity. 
- By considering the workflow of using game and player IDs to include new features into the existing data set, it would allow me to keep my existing data set (and existing work), while giving me the flexibility to include more variables based on the game and player IDs. **In my next entry, I will see if this approach will work for me. and again, see if there are any other learnings that I can gain from this.**

In [25]:
final_table.to_csv("1986_2016_seasons_shifted_v1.csv", index=False)